In [9]:
import xarray as xr
import sys
sys.path.append('/home/563/rl5183/Functions')
import model_functions as func
import os

In [7]:
!ls /g/data/lp01/CMIP6/CMIP

AS-RCEC  CCCma	       E3SM-Project	    IPSL   NASA-GISS  NUIST
AWI	 CCCR-IITM     EC-Earth-Consortium  MIROC  NCAR       SNU
BCC	 CNRM-CERFACS  FIO-QLNM		    MOHC   NCC	      THU
CAMS	 CSIRO	       HAMMOZ-Consortium    MPI-M  NIMS-KMA   UA
CAS	 CSIRO-ARCCSS  INM		    MRI    NOAA-GFDL


In [8]:
# store each section of the directory as a string
institution_dir = '/g/data/lp01/CMIP6/CMIP/'
variable_dir = '/historical/r1i1p1f1/Amon/tas/gr1.5/'
print(institution_dir, variable_dir)

/g/data/lp01/CMIP6/CMIP/ /historical/r1i1p1f1/Amon/tas/gr1.5/


In [17]:
# make a list of all the institutions in the directory
institutions = os.listdir(institution_dir)
type(institutions)

list

In [60]:
#create dictionary
models = {}

for institution in institutions:
    model_list = os.listdir(f'{institution_dir}{institution}')
    for model in model_list:
        # check if the historical model with the right variable exists and if so save the version number for the file
        if os.path.exists(f'{institution_dir}{institution}/{model}{variable_dir}'):
            version = os.listdir(f'{institution_dir}{institution}/{model}{variable_dir}')
            # for each version, call model_path to make the path and then store with the model in a dictionary 'models'
            for v in version:
                path = func.model_path(institution, model, v)
                models[model] = path

In [61]:
len(models)

44

In [39]:
for name, path in models.items():
    print(name, path)

MPI-ESM-1-2-HAM /g/data/lp01/CMIP6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/historical/r1i1p1f1/Amon/tas/gr1.5/v20190627/*.nc
NESM3 /g/data/lp01/CMIP6/CMIP/NUIST/NESM3/historical/r1i1p1f1/Amon/tas/gr1.5/v20190630/*.nc
MCM-UA-1-0 /g/data/lp01/CMIP6/CMIP/UA/MCM-UA-1-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20190731/*.nc
ACCESS-CM2 /g/data/lp01/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/Amon/tas/gr1.5/v20190919/*.nc
MPI-ESM1-2-HR /g/data/lp01/CMIP6/CMIP/MPI-M/MPI-ESM1-2-HR/historical/r1i1p1f1/Amon/tas/gr1.5/v20190710/*.nc
MPI-ESM1-2-LR /g/data/lp01/CMIP6/CMIP/MPI-M/MPI-ESM1-2-LR/historical/r1i1p1f1/Amon/tas/gr1.5/v20190710/*.nc
IPSL-CM6A-LR /g/data/lp01/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/tas/gr1.5/v20180803/*.nc
CAS-ESM2-0 /g/data/lp01/CMIP6/CMIP/CAS/CAS-ESM2-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20200302/*.nc
FGOALS-f3-L /g/data/lp01/CMIP6/CMIP/CAS/FGOALS-f3-L/historical/r1i1p1f1/Amon/tas/gr1.5/v20190927/*.nc
FGOALS-g3 /g/data/lp01/CMIP6/CMIP/CAS/FGOALS-g3/histo

In [40]:
names = []
ds = []

for name, path in models.items():
    try:
        ds.append(xr.open_mfdataset(path, combine='by_coords'))
        names.append(name)
    except OSError:
        # No files read, move on to the next
        continue

In [46]:
type(ds)

list

In [68]:
ds

[<xarray.Dataset>
 Dimensions:    (bnds: 2, lat: 120, lon: 240, time: 1980)
 Coordinates:
   * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
   * lon        (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 354.0 355.5 357.0 358.5
   * lat        (lat) float64 -89.25 -87.75 -86.25 -84.75 ... 86.25 87.75 89.25
     height     float64 ...
 Dimensions without coordinates: bnds
 Data variables:
     time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1980, 2), meta=np.ndarray>
     tas        (time, lat, lon) float32 dask.array<chunksize=(1980, 120, 240), meta=np.ndarray>
 Attributes:
     CDI:                       Climate Data Interface version 1.9.7.1 (http:/...
     history:                   Fri Jan 10 11:57:16 2020: cdo -O -L -P 28 -rem...
     source:                    MPI-ESM1.2-HAM (2017): \naerosol: HAM2.3\natmo...
     institution:               ETH Zurich, Switzerland; Max Planck Institut f...
     Conventions:               CF-1.7 CMIP-6.2
     act

In [43]:
multi_model = xr.concat(ds, dim='model')
multi_model.coords['model'] = names

TypeError: cannot compare cftime.DatetimeNoLeap(1850, 1, 16, 12, 0, 0, 0, 3, 16) and Timestamp('1850-01-16 12:00:00') (different calendars)

In [223]:
model_test = {
    #'MPI-ESM-1-2-HAM': '/g/data/lp01/CMIP6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/historical/r1i1p1f1/Amon/tas/gr1.5/v20190627/*.nc',
    #'NESM3': '/g/data/lp01/CMIP6/CMIP/NUIST/NESM3/historical/r1i1p1f1/Amon/tas/gr1.5/v20190630/*.nc',
    'MCM-UA-1-0': '/g/data/lp01/CMIP6/CMIP/UA/MCM-UA-1-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20190731/*.nc',
    #'ACCESS-CM2' :'/g/data/lp01/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/Amon/tas/gr1.5/v20190919/*.nc',
    #'MPI-ESM1-2-HR': '/g/data/lp01/CMIP6/CMIP/MPI-M/MPI-ESM1-2-HR/historical/r1i1p1f1/Amon/tas/gr1.5/v20190710/*.nc',
    #'MPI-ESM1-2-LR': '/g/data/lp01/CMIP6/CMIP/MPI-M/MPI-ESM1-2-LR/historical/r1i1p1f1/Amon/tas/gr1.5/v20190710/*.nc',
    #'IPSL-CM6A-LR': '/g/data/lp01/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/tas/gr1.5/v20180803/*.nc',
    'CAS-ESM2-0': '/g/data/lp01/CMIP6/CMIP/CAS/CAS-ESM2-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20200302/*.nc',
    'FGOALS-f3-L':'/g/data/lp01/CMIP6/CMIP/CAS/FGOALS-f3-L/historical/r1i1p1f1/Amon/tas/gr1.5/v20190927/*.nc',
    'FGOALS-g3': 'g/data/lp01/CMIP6/CMIP/CAS/FGOALS-g3/historical/r1i1p1f1/Amon/tas/gr1.5/v20190818/*.nc',
    'BCC-ESM1':'/g/data/lp01/CMIP6/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Amon/tas/gr1.5/v20181214/*.nc',
    'BCC-CSM2-MR': '/g/data/lp01/CMIP6/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/Amon/tas/gr1.5/v20181126/*.nc',
    'CIESM' :'/g/data/lp01/CMIP6/CMIP/THU/CIESM/historical/r1i1p1f1/Amon/tas/gr1.5/v20191228/*.nc',
    'TaiESM1': '/g/data/lp01/CMIP6/CMIP/AS-RCEC/TaiESM1/historical/r1i1p1f1/Amon/tas/gr1.5/v20200218/*.nc',
    #'KACE-1-0-G' :'/g/data/lp01/CMIP6/CMIP/NIMS-KMA/KACE-1-0-G/historical/r1i1p1f1/Amon/tas/gr1.5/v20191028/*.nc',
    'GISS-E2-1-G-CC': '/g/data/lp01/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G-CC/historical/r1i1p1f1/Amon/tas/gr1.5/v20190815/*.nc',
    'GISS-E2-1-H': '/g/data/lp01/CMIP6/CMIP/NASA-GISS/GISS-E2-1-H/historical/r1i1p1f1/Amon/tas/gr1.5/v20190403/*.nc',
    'GISS-E2-1-G': '/g/data/lp01/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r1i1p1f1/Amon/tas/gr1.5/v20180827/*.nc',
    #'AWI-CM-1-1-MR': '/g/data/lp01/CMIP6/CMIP/AWI/AWI-CM-1-1-MR/historical/r1i1p1f1/Amon/tas/gr1.5/v20191015/*.nc',
    #'AWI-ESM-1-1-LR': '/g/data/lp01/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/historical/r1i1p1f1/Amon/tas/gr1.5/v20200212/*.nc',
    #'MIROC6': '/g/data/lp01/CMIP6/CMIP/MIROC/MIROC6/historical/r1i1p1f1/Amon/tas/gr1.5/v20181212/*.nc',
    #'MRI-ESM2-0': '/g/data/lp01/CMIP6/CMIP/MRI/MRI-ESM2-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20190222/*.nc',
    'FIO-ESM-2-0': '/g/data/lp01/CMIP6/CMIP/FIO-QLNM/FIO-ESM-2-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20191115/*.nc',
    'CanESM5': '/g/data/lp01/CMIP6/CMIP/CCCma/CanESM5/historical/r1i1p1f1/Amon/tas/gr1.5/v20190429/*.nc',
    #'ACCESS-ESM1-5': '/g/data/lp01/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Amon/tas/gr1.5/v20190922/*.nc',
    'GFDL-CM4': '/g/data/lp01/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/Amon/tas/gr1.5/v20180701/*.nc',
    'GFDL-ESM4': '/g/data/lp01/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Amon/tas/gr1.5/v20190726/*.nc',
    #'E3SM-1-0': '/g/data/lp01/CMIP6/CMIP/E3SM-Project/E3SM-1-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20190913/*.nc',
    'E3SM-1-1': '/g/data/lp01/CMIP6/CMIP/E3SM-Project/E3SM-1-1/historical/r1i1p1f1/Amon/tas/gr1.5/v20191211/*.nc',
    'E3SM-1-1-ECA': '/g/data/lp01/CMIP6/CMIP/E3SM-Project/E3SM-1-1-ECA/historical/r1i1p1f1/Amon/tas/gr1.5/v20200127/*.nc',
    'INM-CM4-8': '/g/data/lp01/CMIP6/CMIP/INM/INM-CM4-8/historical/r1i1p1f1/Amon/tas/gr1.5/v20190530/*.nc',
    'INM-CM5-0': '/g/data/lp01/CMIP6/CMIP/INM/INM-CM5-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20190610/*.nc',
    #'EC-Earth3-Veg-LR': '/g/data/lp01/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/historical/r1i1p1f1/Amon/tas/gr1.5/v20200217/*.nc',
    #'EC-Earth3-Veg': '/g/data/lp01/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r1i1p1f1/Amon/tas/gr1.5/v20200225/*.nc',
    #'EC-Earth3': '/g/data/lp01/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/historical/r1i1p1f1/Amon/tas/gr1.5/v20200310/*.nc',
    'NorCPM1': '/g/data/lp01/CMIP6/CMIP/NCC/NorCPM1/historical/r1i1p1f1/Amon/tas/gr1.5/v20190914/*.nc',
    'NorESM2-LM': '/g/data/lp01/CMIP6/CMIP/NCC/NorESM2-LM/historical/r1i1p1f1/Amon/tas/gr1.5/v20190815/*.nc',
    'NorESM2-MM': '/g/data/lp01/CMIP6/CMIP/NCC/NorESM2-MM/historical/r1i1p1f1/Amon/tas/gr1.5/v20191108/*.nc',
    'CAMS-CSM1-0': '/g/data/lp01/CMIP6/CMIP/CAMS/CAMS-CSM1-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20190708/*.nc',
    #'CESM2': '/g/data/lp01/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gr1.5/v20190308/*.nc',
    #CESM2-WACCM': '/g/data/lp01/CMIP6/CMIP/NCAR/CESM2-WACCM/historical/r1i1p1f1/Amon/tas/gr1.5/v20190227/*.nc',
    #CESM2-FV2': '/g/data/lp01/CMIP6/CMIP/NCAR/CESM2-FV2/historical/r1i1p1f1/Amon/tas/gr1.5/v20191120/*.nc',
    #'CESM2-WACCM-FV2': '/g/data/lp01/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/historical/r1i1p1f1/Amon/tas/gr1.5/v20191120/*.nc',
    'SAM0-UNICON': '/g/data/lp01/CMIP6/CMIP/SNU/SAM0-UNICON/historical/r1i1p1f1/Amon/tas/gr1.5/v20190323/*.nc',
}

In [224]:
names2 = []
ds2 = []

for name, path in model_test.items():
    try:
        ds2.append(xr.open_mfdataset(path, combine='by_coords'))
        names2.append(name)
    except OSError:
        # No files read, move on to the next
        continue

In [225]:
multi_model = xr.concat(ds2, dim='model')
multi_model.coords['model'] = names2

In [226]:
multi_model

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 120, lon: 240, model: 22, time: 1980)
Coordinates:
    height     float64 2.0
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lat        (lat) float64 -89.25 -87.75 -86.25 -84.75 ... 86.25 87.75 89.25
  * lon        (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 354.0 355.5 357.0 358.5
  * model      (model) <U14 'CAS-ESM2-0' 'FGOALS-f3-L' ... 'SAM0-UNICON'
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (model, time, bnds) object dask.array<chunksize=(1, 1980, 2), meta=np.ndarray>
    tas        (model, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 120, 240), meta=np.ndarray>
Attributes:
    CDI:                       Climate Data Interface version 1.9.8 (https://...
    Conventions:               CF-1.7 CMIP-6.2
    history:                   Mon Apr 06 10:08:24 2020: cdo -O -L -P 28 -rem...
    source:                    CAS-ESM 2.0 (2019): \naerosol: IAP AACM\natmos...
    institution:               Chinese Academy of Sciences, Beijing 100029, C...
    activity_id:               CMIP
    branch_method:             standard
    branch_time_in_child:      0.0
    branch_time_in_parent:     0.0
    contact:                   Zhang He (zhanghe@mail.iap.ac.cn)
    creation_date:             2020-03-02T12:28:26Z
    data_specs_version:        01.00.31
    experiment:                all-forcing simulation of the recent past
    experiment_id:             historical
    external_variables:        areacella
    forcing_index:             1
    frequency:                 mon
    further_info_url:          https://furtherinfo.es-doc.org/CMIP6.CAS.CAS-E...
    grid:                      native atmosphere regular grid (128x256 latxlon)
    grid_label:                gn
    initialization_index:      1
    institution_id:            CAS
    mip_era:                   CMIP6
    nominal_resolution:        100 km
    parent_activity_id:        CMIP
    parent_experiment_id:      piControl
    parent_mip_era:            CMIP6
    parent_source_id:          CAS-ESM2-0
    parent_time_units:         days since 1850-01-01
    parent_variant_label:      r1i1p1f1
    physics_index:             1
    product:                   model-output
    realization_index:         1
    realm:                     atmos
    run_variant:               3rd realization
    source_id:                 CAS-ESM2-0
    source_type:               AOGCM
    sub_experiment:            none
    sub_experiment_id:         none
    table_id:                  Amon
    table_info:                Creation Date:(24 July 2019) MD5:b9834a2d0728c...
    title:                     CAS-ESM2-0 output prepared for CMIP6
    tracking_id:               hdl:21.14100/22e89a1b-f73e-45be-84dc-7d0aabbeea9d
    variable_id:               tas
    variant_label:             r1i1p1f1
    license:                   CMIP6 model data produced by Institute of Atmo...
    cmor_version:              3.5.0
    CDO:                       Climate Data Operators version 1.9.8 (https://...
    cdo_openmp_thread_number:  28